In [16]:
from pyspark import SparkConf, SparkContext
sc = SparkContext.getOrCreate(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        words = [word.lower() for word in words]
        words.append("END")
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article).flatMap(lambda x:x)

indexed = wiki.zipWithIndex().map(lambda (k,v):(v,k))
shifted = indexed.map(lambda (k,v):(k+1,v))
joined = shifted.join(indexed)
joined.cache()

PythonRDD[77] at RDD at PythonRDD.scala:48

In [17]:
filtered = joined.values().filter(lambda v:v[0] == "narodnaya" and v[1] != "END")
print(filtered.map(lambda v:("%s_%s"%(v[0],v[1]),1)).reduceByKey(lambda x,y:x+y).collect())


[(u'narodnaya_gazeta', 1), (u'narodnaya_volya', 9)]
